In [187]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.layers import Conv2D, UpSampling2D, InputLayer, Lambda
import tensorflow.keras.backend as K

In [188]:
vgg19_cl = keras.applications.VGG19()
vgg19_cl.trainable = False
vgg19_cl.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_98 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [189]:
model_inp = keras.Sequential(name="conv_imag")
model_inp.add(InputLayer(input_shape=(224, 224, 1)))
model_inp.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model_inp.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model_inp.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model_inp.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model_inp.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model_inp.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model_inp.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model_inp.add(Conv2D(1024, (3, 3), activation='relu', padding='same'))
model_inp.add(Conv2D(1024, (3, 3), activation='relu', padding='same'))
model_inp.summary()

Model: "conv_imag"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_120 (Conv2D)         (None, 224, 224, 64)      1792      
                                                                 
 conv2d_121 (Conv2D)         (None, 112, 112, 64)      36928     
                                                                 
 conv2d_122 (Conv2D)         (None, 112, 112, 128)     73856     
                                                                 
 conv2d_123 (Conv2D)         (None, 56, 56, 128)       147584    
                                                                 
 conv2d_124 (Conv2D)         (None, 56, 56, 256)       295168    
                                                                 
 conv2d_125 (Conv2D)         (None, 28, 28, 256)       590080    
                                                                 
 conv2d_126 (Conv2D)         (None, 28, 28, 512)       11

In [190]:
model_out = keras.Sequential()
model_out.add(InputLayer(input_shape=(28, 28, 1025)))
model_out.add(UpSampling2D((2, 2)))
model_out.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model_out.add(UpSampling2D((2, 2)))
model_out.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model_out.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model_out.add(UpSampling2D((2, 2)))
model_out.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_129 (Conv2D)         (None, 28, 28, 1026)      9475110   
                                                                 
 conv2d_130 (Conv2D)         (None, 28, 28, 512)       4728320   
                                                                 
 up_sampling2d_41 (UpSamplin  (None, 56, 56, 512)      0         
 g2D)                                                            
                                                                 
 conv2d_131 (Conv2D)         (None, 56, 56, 256)       1179904   
                                                                 
 up_sampling2d_42 (UpSamplin  (None, 112, 112, 256)    0         
 g2D)                                                            
                                                                 
 conv2d_132 (Conv2D)         (None, 112, 112, 64)    

In [191]:
inp_conv = InputLayer((224, 224, 1))


def func(x1, x2):
    """

    :param x1:  shape = (28, 28, 1024)
    :param x2: shape = (1000, )
    :return: x3 shape (28, 28, 1026)
    """
    temp_zero = np.zeros(shape=(28*28*2))
    temp_zero[:1000] = x2
    x2 = np.reshape(temp_zero,newshape=(28,28,2))
    return np.reshape(np.append(x1,x2),newshape=(28,28,1026))

x1 = np.array([[[i for i in range(1024)] for j in range(28)]for _ in range(28)])
x2 = np.array([0 if i != 10 else 1 for i in range(1000)])


func(x1,x2).shape



Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_101 (InputLayer)         [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 vgg19 (Functional)             (None, 1000)         143667240   ['input_101[0][0]']              
                                                                                                  
 concatenate_7 (Concatenate)    (1, 1568)            0           ['vgg19[0][0]']                  
                                                                                                  
 reshape_3 (Reshape)            (1, 28, 28, 2)       0           ['concatenate_7[0][0]']    